In [ ]:
# imports
import os
import pandas as pd

## Data Loading

## Preprocessing

We have multiple labels per observation.

* Atelectasis
* Cardiomegaly
* Effusion
* Infiltration
* Mass
* Nodule
* Pneumonia
* Pneumothorax
* Consolidation
* Edema
* Emphysema
* Fibrosis
* Pleural_Thickening
* Hernia

"Mass" is difficult to detect, therefore if there any "multiple-diagnoses" with this label, we will make the observation solely "Mass". All other multiple-diagnoses will be disregarded.